# Data 690 Practical AI: Final Project
Jessica Conroy
## Notebook 2: Training the classifier
Contents:
- <b>Notebook 1</b> in this series collects the dataset for the project. 
- <b>Notebook 2</b> takes the next step by preparing and transforming the data for use in a pytorch model. It includes the training of the model with updatable constants for testing different parameters and configurations.
- <b>Notebook 3</b> This is the code used in the App.py script below, but with additional documentation added
- <b>App.py</b> represents the 3rd piece of the project which will launch the sign language detector. 

## Comments:
Earlier in the life of this project, I did initially attempt training using a Faster RCNN. That notebook is available for reference in the notebooks file. I also attempted using tensorflow to train this model, but found that it was a little to difficult to get working within the timeframe of this project.
Performance was extremely poor when using it in the final application and I therefore changed tactics moving forward. 

## Sources:
### Guides:
https://pyimagesearch.com/2021/11/01/training-an-object-detector-from-scratch-in-pytorch/

https://medium.com/academy-eldoradocps/creating-a-custom-neural-network-with-pytorch-fd3621705d32

https://github.com/Gunnika/Sign_Language_Detector-PyTorch

https://medium.com/bitgrit-data-science-publication/building-an-image-classification-model-with-pytorch-from-scratch-f10452073212

https://github.com/EdwardRaff/Inside-Deep-Learning

Raff, E. (2022). Inside Deep Learning: Math, Algorithms, Models (Annotated ed.). Manning.

https://towardsdatascience.com/custom-dataset-in-pytorch-part-1-images-2df3152895


### Documentation:
https://github.com/pytorch/vision/blob/main/torchvision/models/resnet.py

https://pytorch.org/vision/stable/models.html

https://pytorch.org/tutorials/beginner/hyperparameter_tuning_tutorial.html

https://pytorch.org/docs/stable/generated/torch.nn.Identity.html

https://pytorch.org/tutorials/beginner/saving_loading_models.html

In [54]:
import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
from torch.optim import Adam
from torchvision.models import resnet34, resnet50, resnet101
from torch.nn import CrossEntropyLoss
from torch.nn import Dropout
from torch.nn import Identity
from torch.nn import Linear
from torch.nn import Module
from torch.nn import ReLU
from torch.nn import Sequential
from torch.nn import Sigmoid
from torch.nn import Flatten
import torch.nn as nn
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import cv2
import glob
import numpy
import random
import xml.etree.ElementTree as ET

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import matplotlib.pyplot as plt
import os

import time

os.getcwd()


# !unzip "collectedimgs.zip" -d "./data"
!unzip "collectedimgs (2).zip" -d "./data" 

Archive:  collectedimgs (2).zip
replace ./data/collectedimgs/test/a.ee50b36b-cccd-11ec-ab81-287fcffe570f.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


In [132]:
# os.mkdir('annotations')
# os.mkdir("output")
# os.chdir('output')
# os.mkdir("plots")
# os.chdir('..')
BASE_PATH = ""
# define the path to the base output directory
BASE_OUTPUT = "output"
# define the path to the output model, label encoder, plots output
# directory, and testing image paths
MODEL_PATH = os.path.sep.join([BASE_OUTPUT, "classification_200epochs_croppedTI.pth"])
Checkpoint = os.path.sep.join([BASE_OUTPUT, "classification_200epochs_croppedTI_checkpoint.pth"])
LE_PATH = os.path.sep.join([BASE_OUTPUT, "le.pickle"])
PLOTS_PATH = os.path.sep.join([BASE_OUTPUT, "plots"])
TEST_PATHS = os.path.sep.join([BASE_OUTPUT, "test_paths.txt"])

In [154]:
# determine the current device and based on that set the pin memory
# flag
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
PIN_MEMORY = True if DEVICE == "cuda" else False
# specify ImageNet mean and standard deviation
MEAN = [0.485, 0.456, 0.406]
STD = [0.229, 0.224, 0.225]
# initialize our initial learning rate, number of epochs to train
# for, and the batch size
INIT_LR = 1e-4
NUM_EPOCHS = 200
BATCH_SIZE = 32
# specify the loss weights
LABELS = 1.0
BBOX = 1.0

In [155]:
train_data_path = './data/collectedimgs/train' 
test_data_path = './data/collectedimgs/test'


train_image_paths = []
train_annotation_paths = []
test_image_paths = []
test_annotations_paths = []

train_annotation_paths = glob.glob(train_data_path+"/*.xml")
train_annotation_paths = sorted(train_annotation_paths)
train_image_paths = glob.glob(train_data_path+"/*.jpg")
train_image_paths = sorted(train_image_paths)

test_annotations_paths = glob.glob(test_data_path+"/*.xml")
test_annotations_paths = sorted(test_annotations_paths)
test_image_paths = glob.glob(test_data_path+"/*.jpg")
test_image_paths = sorted(test_image_paths)

classes = []
for path in train_image_paths:
    filename = path.split('/')[-1]
    class_name = filename.split('.')[0]
    if class_name == 'ThankYou':
        class_name = 'thank you'
    if class_name in classes:
        continue
    else:
        classes.append(class_name)

print(classes)

['thank you', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'hello', 'i', 'j', 'k', 'l', 'm', 'my', 'n', 'name', 'no', 'o', 'p', 'project', 'q', 'r', 's', 't', 'this', 'u', 'v', 'w', 'x', 'y', 'yes', 'z']


In [156]:
# len(test_image_paths_lite)

In [157]:
#Uncomment to select only a few classes to train

my_classes = ['ThankYou', 'hello','my', 'name', 'project','this', 'j', 'e', 's']

train_annotation_paths_lite = []
test_annotation_paths_lite = []

train_image_paths_lite = []
test_image_paths_lite = []

for i,path in enumerate(train_image_paths):
    filename = path.split('/')[-1]
    class_name = filename.split('.')[0]
    for CN in my_classes:
        if CN == class_name:
            train_image_paths_lite.append(path)
            train_annotation_paths_lite.append(train_annotation_paths[i])

for i,path in enumerate(test_image_paths):
    filename = path.split('/')[-1]
    class_name = filename.split('.')[0]
    for CN in my_classes:
        if CN == class_name:
            test_image_paths_lite.append(path)
            test_annotation_paths_lite.append(test_annotations_paths[i])

classes = []
for path in train_annotation_paths_lite:
    filename = path.split('/')[-1]
    class_name = filename.split('.')[0]
    if class_name == 'ThankYou':
        class_name = 'thank you'
    if class_name in classes:
        continue
    else:
        classes.append(class_name)

In [158]:
print(train_annotation_paths_lite)
print(train_image_paths_lite)

['./data/collectedimgs/train/ThankYou.4bd68bea-ccd8-11ec-b32d-287fcffe570f.xml', './data/collectedimgs/train/ThankYou.4da5dcd5-ccd8-11ec-bcf7-287fcffe570f.xml', './data/collectedimgs/train/ThankYou.4f748e67-ccd8-11ec-bafb-287fcffe570f.xml', './data/collectedimgs/train/ThankYou.5315c7a4-ccd8-11ec-ae63-287fcffe570f.xml', './data/collectedimgs/train/ThankYou.54e65b5d-ccd8-11ec-b94a-287fcffe570f.xml', './data/collectedimgs/train/ThankYou.56b93aaa-ccd8-11ec-ae0a-287fcffe570f.xml', './data/collectedimgs/train/ThankYou.58859e9b-ccd8-11ec-a668-287fcffe570f.xml', './data/collectedimgs/train/ThankYou.5a55f747-ccd8-11ec-8893-287fcffe570f.xml', './data/collectedimgs/train/ThankYou.5c2735bd-ccd8-11ec-a2f7-287fcffe570f.xml', './data/collectedimgs/train/ThankYou.5df665d4-ccd8-11ec-97c6-287fcffe570f.xml', './data/collectedimgs/train/ThankYou.5fc525ba-ccd8-11ec-9f83-287fcffe570f.xml', './data/collectedimgs/train/ThankYou.619287c2-ccd8-11ec-902f-287fcffe570f.xml', './data/collectedimgs/train/ThankYou.63

In [159]:
classes

['thank you', 'e', 'hello', 'j', 'my', 'name', 'project', 's', 'this']

In [160]:
split = train_test_split(train_image_paths_lite, train_annotation_paths_lite, test_size=0.10, random_state=42)
(train_image_paths, valid_image_paths) = split[:2]
(train_annotation_paths, valid_annotation_paths) = split[2:4]

print("Train size: {}\nValid size: {}\nTest size: {}".format(len(train_image_paths), len(valid_image_paths), len(test_image_paths_lite)))
print("Train size: {}\nValid size: {}\nTest size: {}".format(len(train_annotation_paths), len(valid_annotation_paths), len(test_annotation_paths_lite)))


Train size: 292
Valid size: 33
Test size: 36
Train size: 292
Valid size: 33
Test size: 36


In [161]:
idx_to_class = {i:j for i, j in enumerate(classes)}
class_to_idx = {value:key for key,value in idx_to_class.items()}
class_to_idx

{'e': 1,
 'hello': 2,
 'j': 3,
 'my': 4,
 'name': 5,
 'project': 6,
 's': 7,
 'thank you': 0,
 'this': 8}

In [162]:
from torch.utils.data import Dataset

train_transforms = transforms.Compose([
    transforms.ToPILImage(),
    transforms.ToTensor(),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.Normalize(mean=MEAN, std=STD)
])

test_transforms = transforms.Compose([
    transforms.ToPILImage(),
    transforms.ToTensor(),
    transforms.Normalize(mean=MEAN, std=STD)
])


class ASLDataset(Dataset):
    def __init__(self, image_paths, xml_paths, transform=False):
        self.image_paths = image_paths
        self.annotations = xml_paths
        self.transform = transform
        
    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        annotation_path = self.annotations[idx]
        root = ET.parse(annotation_path).getroot()
        xmin = int(root.find("object").find("bndbox").find("xmin").text)
        ymin = int(root.find("object").find("bndbox").find("ymin").text)
        xmax = int(root.find("object").find("bndbox").find("xmax").text)
        ymax = int(root.find("object").find("bndbox").find("ymax").text)
        class_name = root.find("object").find("name").text.lower()
        
        image_filepath = self.image_paths[idx]
        image = cv2.imread(image_filepath)
        image = image[ymin:ymax, xmin:xmax] #crop to just hands
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = cv2.resize(image, (224, 224))

        label =  class_name
        label = class_to_idx[label]
        if self.transform is not None:
            image = self.transform(image)
        
        return image, label

In [163]:
train_dataset = ASLDataset(train_image_paths,train_annotation_paths, train_transforms)
valid_dataset = ASLDataset(valid_image_paths,valid_annotation_paths, test_transforms) #test transforms are applied
test_dataset = ASLDataset(test_image_paths_lite,test_annotation_paths_lite, test_transforms)

In [164]:
# train_dataset[0]
trainSteps = len(train_dataset) // BATCH_SIZE
valSteps = len(valid_dataset) // BATCH_SIZE

In [165]:
train_loader = DataLoader(
    train_dataset, batch_size=BATCH_SIZE, shuffle=True
)

valid_loader = DataLoader(
    valid_dataset, batch_size=BATCH_SIZE, shuffle=True
)


test_loader = DataLoader(
    test_dataset, batch_size=BATCH_SIZE, shuffle=False
)

In [166]:
class ObjectClassifier(Module):
    def __init__(self, baseModel, numClasses):
        super(ObjectClassifier, self).__init__()
        # initialize the base model and the number of classes
        self.baseModel = baseModel
        self.numClasses = numClasses

        # build the classifier head to predict the class labels
        self.classifier = Sequential(
            Linear(baseModel.fc.in_features, 512),
            ReLU(),
            Dropout(),
            Linear(512, 512),
            ReLU(),
            Dropout(),
            Linear(512, self.numClasses)
            )
        # set the classifier of our base model to produce outputs
        # from the last convolution block
        self.baseModel.fc = Identity()
    def forward(self, x):
    # pass the inputs through the base model and then obtain
    # predictions from two different branches of the network
        features = self.baseModel(x)
        classLogits = self.classifier(features)
    # return the outputs as a tuple
        return (classLogits)

In [167]:
# from torchvision.models import resnet34
# resnet = resnet34(pretrained=True)
resnet = resnet50(pretrained=True)
# resnet = resnet101(pretrained=True)
# resnet = resnet18(pretrained=True)
# rcnn = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
# freeze all ResNet50 layers so they will *not* be updated during the
# training process
for param in resnet.parameters():
    param.requires_grad = False

In [168]:
ObjectClassifier = ObjectClassifier(resnet, len(idx_to_class))
ObjectClassifier = ObjectClassifier.to(DEVICE)
# define our loss functions
classLossFunc = CrossEntropyLoss()
# bboxLossFunc = MSELoss()
# initialize the optimizer, compile the model, and show the model
# summary
opt = Adam(ObjectClassifier.parameters(), lr=INIT_LR)
# opt = torch.optim.SGD(ObjectClassifier.parameters(), lr=.1, momentum=0.9)
print(ObjectClassifier)
# initialize a dictionary to store training history
H = {"total_train_loss": [], "total_val_loss": [], "train_class_acc": [],"val_class_acc": []}

ObjectClassifier(
  (baseModel): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequential(

In [169]:
def moveTo(obj, device):
    """
    obj: the python object to move to a device, or to move its contents to a device
    device: the compute device to move objects to
    """
    if hasattr(obj, "to"):
        return obj.to(device)
    elif isinstance(obj, list):
        return [moveTo(x, device) for x in obj]
    elif isinstance(obj, tuple):
        return tuple(moveTo(list(obj), device))
    elif isinstance(obj, set):
        return set(moveTo(list(obj), device))
    elif isinstance(obj, dict):
        to_ret = dict()
        for key, value in obj.items():
            to_ret[moveTo(key, device)] = moveTo(value, device)
        return to_ret
    else:
        return obj

In [170]:
print("[INFO] training the network...")
startTime = time.time()
for e in tqdm(range(NUM_EPOCHS)):
    # set the model in training mode
    ObjectClassifier.train()
    # initialize the total training and validation loss
    totalTrainLoss = 0
    totalValLoss = 0
    # initialize the number of correct predictions in the training
    # and validation step
    trainCorrect = 0
    valCorrect = 0
    # loop over the training set
    for (images, labels) in train_loader:
        # send the input to the device
        # print(labels)
        images = moveTo(images, DEVICE)
        labels = moveTo(labels, DEVICE)
        # perform a forward pass and calculate the training loss
        predictions = ObjectClassifier(images)
        classLoss = classLossFunc(predictions, labels)
        totalLoss = (LABELS * classLoss)
        # zero out the gradients, perform the backpropagation step,
        # and update the weights
        opt.zero_grad()
        totalLoss.backward()
        opt.step()
        # add the loss to the total training loss so far and
        # calculate the number of correct predictions
        totalTrainLoss += totalLoss
        trainCorrect += (predictions.argmax(1) == labels).type(
            torch.float).sum().item()
    # switch off autograd
    with torch.no_grad():
        # set the model in evaluation mode
        ObjectClassifier.eval()
        # loop over the validation set
        for (images, labels) in valid_loader:
            # send the input to the device
            images = moveTo(images, DEVICE)
            labels = moveTo(labels, DEVICE)
            # make the predictions and calculate the validation loss
            predictions = ObjectClassifier(images)
            classLoss = classLossFunc(predictions, labels)
            totalLoss = (LABELS * classLoss)
            totalValLoss += totalLoss
            # calculate the number of correct predictions
            valCorrect += (predictions.argmax(1) == labels).type(
                torch.float).sum().item()
    # calculate the average training and validation loss
    avgTrainLoss = totalTrainLoss / trainSteps
    avgValLoss = totalValLoss / valSteps
    # calculate the training and validation accuracy
    trainCorrect = trainCorrect / len(train_dataset)
    valCorrect = valCorrect / len(test_dataset)
    # update our training history
    H["total_train_loss"].append(avgTrainLoss.cpu().detach().numpy())
    H["train_class_acc"].append(trainCorrect)
    H["total_val_loss"].append(avgValLoss.cpu().detach().numpy())
    H["val_class_acc"].append(valCorrect)
    # print(avgValLoss.item())
    # print(min(H["total_val_loss"]))
    # print(avgValLoss.item() < min(H["total_val_loss"]))
    if avgValLoss.item() == min(H["total_val_loss"]):#save best path
        print(avgValLoss)
        print(min(H["total_val_loss"]))
        torch.save({
            'epoch': e,
            'model_state_dict': ObjectClassifier.state_dict(),
            'optimizer_state_dict': opt.state_dict(),
            'results' : H
            }, Checkpoint)
    # print the model training and validation information
    print("[INFO] EPOCH: {}/{}".format(e + 1, NUM_EPOCHS))
    print("Train loss: {:.6f}, Train accuracy: {:.4f}".format(
        avgTrainLoss, trainCorrect))
    print("Val loss: {:.6f}, Val accuracy: {:.4f}".format(
        avgValLoss, valCorrect))
endTime = time.time()
print("[INFO] total time taken to train the model: {:.2f}s".format(endTime - startTime))


[INFO] training the network...


  0%|          | 0/200 [00:00<?, ?it/s]

tensor(4.3772, device='cuda:0')
4.3771973


  0%|          | 1/200 [00:02<09:04,  2.74s/it]

[INFO] EPOCH: 1/200
Train loss: 2.440855, Train accuracy: 0.1336
Val loss: 4.377197, Val accuracy: 0.1944
tensor(4.2482, device='cuda:0')
4.2481775


...

[INFO] EPOCH: 198/200
Train loss: 0.056805, Train accuracy: 0.9932
Val loss: 0.132647, Val accuracy: 0.8889


100%|█████████▉| 199/200 [08:07<00:02,  2.35s/it]

[INFO] EPOCH: 199/200
Train loss: 0.054404, Train accuracy: 0.9932
Val loss: 0.109157, Val accuracy: 0.8889


100%|██████████| 200/200 [08:10<00:00,  2.45s/it]

[INFO] EPOCH: 200/200
Train loss: 0.188333, Train accuracy: 0.9897
Val loss: 0.107064, Val accuracy: 0.8889
[INFO] total time taken to train the model: 490.30s


In [172]:
from google.colab import files
files.download('/content/output/classification_200epochs_croppedTI_checkpoint.pth')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [171]:
testSteps = len(test_dataset) // BATCH_SIZE
totaltestLoss = 0
testCorrect = 0
with torch.no_grad():
        # set the model in evaluation mode
        ObjectClassifier.eval()
        # loop over the validation set
        for (images, labels) in test_loader:
            # send the input to the device
            images = moveTo(images, DEVICE)
            labels = moveTo(labels, DEVICE)
            # make the predictions and calculate the validation loss
            predictions = ObjectClassifier(images)
            classLoss = classLossFunc(predictions, labels)
            totalLoss = (LABELS * classLoss)
            totaltestLoss += totalLoss
            # calculate the number of correct predictions
            testCorrect += (predictions.argmax(1) == labels).type(
                torch.float).sum().item()

avgtestloss = totaltestLoss / testSteps
# calculate the training and validation accuracy
testCorrect = testCorrect / len(test_dataset)
print("Test loss: {:.6f}, Test accuracy: {:.4f}".format(
        avgtestloss, testCorrect))

Test loss: 0.032765, Test accuracy: 1.0000


In [ ]:
#resnet50
#Test loss: 0.064608, Test accuracy: 0.9722
#resnet101
# Test loss: 2.753026, Test accuracy: 0.8611
#resnet34
# Test loss: 0.072158, Test accuracy: 0.9722
#resnet50, batch = 32
#Test loss: 0.055216, Test accuracy: 1.0000
#resnet50, optimizer = SDG